In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import numpy as np
import shapely.geometry as shpg
from shapely.ops import linemerge
from shapely import set_precision
import os
import sys
import csv
import json
import logging
import shutil
import utm

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
sys.path.append(os.getcwd() + '/../..')
from utils import mkdir, open_zip_shapefile, open_tar_shapefile, haversine, correct_geoms, fix_overaps

In [ ]:
log = logging.getLogger('papermill')
logging.basicConfig(level='INFO', format="%(message)s")

## Files and storage paths

In [ ]:
# Region of interest
reg = 1

# go down from rgi7_scripts/workflow
data_dir = '../../../../rgi7_data/'

# Centerlines from OGGM
cls_dir = f'{data_dir}/l6_centerlines/output/'

# Input dirctory
input_dir = os.path.join(data_dir, 'l5_rgi7b1')

# Output directories
output_dir = mkdir(os.path.join(data_dir, 'l7_rgi7bf'))
output_dir_points = mkdir(os.path.join(data_dir, 'l6_terminus_points'))
output_dir_zip = mkdir(os.path.join(data_dir, 'l7_rgi7bf_zip'))

In [ ]:
# Parameters
reg = 10


In [ ]:
reg_str = f'{reg:02d}'

In [ ]:
reg_file = os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O1Regions')
reg_file = gpd.read_file(reg_file)
reg_file = reg_file.loc[reg_file.o1region == f'{reg:02d}'].iloc[0]

input_dir_reg = f'{input_dir}/RGI2000-v7.0-G-{reg_file.long_code}/'
output_dir_reg = mkdir(f'{output_dir}/RGI2000-v7.0-G-{reg_file.long_code}/', reset=True)
output_dir_cl = mkdir(f'{output_dir}/RGI2000-v7.0-L-{reg_file.long_code}/', reset=True)

### Copy directory Glacier product

Many things are unchanged:

In [ ]:
from distutils.dir_util import copy_tree
copy_tree(input_dir_reg, output_dir_reg)

### Load the shp data to overwrite

In [ ]:
# Read files
rgi_reg_df = gpd.read_file(f'{input_dir_reg}/RGI2000-v7.0-G-{reg_file.long_code}.shp')

## Load the centerline data

In [ ]:
fp = f'{cls_dir}/centerlines_rgi{reg_str}.tar.gz'
cls = open_tar_shapefile(fp)

In [ ]:
with open('../rgi7_centerline_attributes_metadata.json', 'r') as infile:
    meta_cls_product = json.load(infile)

In [ ]:
rename = {
    'RGIID':'rgi_g_id',
    'SEGMENT_ID':'segment_id',
    'STRAHLER':'strahler_n',
    'OUTFLOW_ID':'outflow_id',
    'LE_SEGMENT':'length_m',
    'MAIN':'is_main',
}

In [ ]:
cls = cls.rename(rename, axis=1)
cls['rgi_id'] = ''

In [ ]:
cls_new = cls[[]].copy().reset_index(drop=True)
for col, attrs in meta_cls_product.items():
    assert col in cls
    if attrs['datatype']:
        cls_new[col] = cls[col].astype(attrs['datatype'])
    else: 
        cls_new[col] = cls[col]

In [ ]:
assert len(cls_new) < 99999

In [ ]:
assert cls_new['strahler_n'].min() == 0
cls_new['strahler_n'] += 1  # they actually begin at 1.

In [ ]:
cls_new = gpd.GeoDataFrame(cls_new)
cls_new.crs = cls.crs

### Pick the new term locs and length for the glacier product

In [ ]:
cls_main = cls_new.loc[cls_new.is_main == 1]
assert len(cls_main) == len(rgi_reg_df)
assert np.alltrue(cls_main['rgi_g_id'].values == rgi_reg_df['rgi_id'])

In [ ]:
rgi_reg_df['lmax_m'] = cls_main['length_m'].values

In [ ]:
term_lon = []
term_lat = []
for geom in cls_main.geometry:
    x, y = geom.coords.xy
    term_lon.append(x[-1])
    term_lat.append(y[-1])

In [ ]:
rgi_reg_df['termlon'] = term_lon
rgi_reg_df['termlat'] = term_lat

In [ ]:
topoints = rgi_reg_df.copy()
topoints['geometry'] = [shpg.Point(x, y) for x, y in zip(rgi_reg_df['termlon'], rgi_reg_df['termlat'])]
topoints.to_file(f'{output_dir_points}/terminus_points_rgi{reg_str}.shp.zip', driver='ESRI Shapefile')

### RGI Ids centerlines 

In [ ]:
baseid = f'RGI2000-v7.0-L-{reg:02d}-'
cls_new['rgi_id'] = [baseid + f'{l+1:05d}' for l in range(len(cls_new))]

## Write out and zip

### Updated glacier product 

In [ ]:
group_dir = f'{output_dir_zip}/RGI2000-v7.0-G-global/'
mkdir(group_dir)

In [ ]:
print('Writing...')
rgi_reg_df.to_file(output_dir_reg + f'/RGI2000-v7.0-G-{reg_file.long_code}.shp')
rgi_reg_df.drop('geometry', axis=1).set_index('rgi_id').to_csv(output_dir_reg + f'/RGI2000-v7.0-G-{reg_file.long_code}-attributes.csv', quoting=csv.QUOTE_NONNUMERIC)
shutil.copyfile('../rgi7_attributes_metadata.json', output_dir_reg + f'/RGI2000-v7.0-G-{reg_file.long_code}-attributes_metadata.json')
shutil.copyfile('../README_tpl.md', output_dir_reg + f'/README.md')

print('Zipping...')
shutil.make_archive(f'{group_dir}/RGI2000-v7.0-G-{reg_file.long_code}', 'zip', output_dir_reg)

### Centerline product

In [ ]:
group_dir = f'{output_dir_zip}/RGI2000-v7.0-L-global/'
mkdir(group_dir)

In [ ]:
print('Writing...')
cls_new.to_file(output_dir_cl + f'/RGI2000-v7.0-L-{reg_file.long_code}.shp')
cls_new.drop('geometry', axis=1).set_index('rgi_id').to_csv(output_dir_cl + f'/RGI2000-v7.0-L-{reg_file.long_code}-attributes.csv', quoting=csv.QUOTE_NONNUMERIC)
shutil.copyfile('../rgi7_centerline_attributes_metadata.json', output_dir_cl + f'/RGI2000-v7.0-L-{reg_file.long_code}-attributes_metadata.json')
shutil.copyfile('../README_tpl.md', output_dir_cl + f'/README.md')

print('Zipping...')
shutil.make_archive(f'{group_dir}/RGI2000-v7.0-L-{reg_file.long_code}', 'zip', output_dir_cl)

### Rezip complex and intersects product

In [ ]:
group_dir = f'{output_dir_zip}/RGI2000-v7.0-C-global/'
mkdir(group_dir)

In [ ]:
dir_to_zip = os.path.join(data_dir, 'l4_rgi7b0', f'RGI2000-v7.0-C-{reg_file.long_code}')
# Overwrite README
shutil.copyfile('../README_tpl.md', dir_to_zip + f'/README.md')
print('Zipping...')
shutil.make_archive(f'{group_dir}/RGI2000-v7.0-C-{reg_file.long_code}', 'zip', dir_to_zip)

In [ ]:
group_dir = f'{output_dir_zip}/RGI2000-v7.0-I-global/'
mkdir(group_dir)

In [ ]:
dir_to_zip = os.path.join(data_dir, 'l4_rgi7b0', f'RGI2000-v7.0-I-{reg_file.long_code}')
# Overwrite README
shutil.copyfile('../README_tpl.md', dir_to_zip + f'/README.md')
print('Zipping...')
shutil.make_archive(f'{group_dir}/RGI2000-v7.0-I-{reg_file.long_code}', 'zip', dir_to_zip)

### Region files

In [ ]:
shutil.copyfile(os.path.join(data_dir, 'l5_rgi7b1_zip', 'RGI2000-v7.0-regions.zip'), 
                os.path.join(output_dir_zip, 'RGI2000-v7.0-regions.zip'))